In [1]:
# load the library
import numpy as np
import pandas as pd
from tqdm import tqdm
import sqlite3

# load selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pyperclip
from IPython.display import clear_output

In [2]:
# load the library
import requests
import bs4
from bs4 import BeautifulSoup
import requests
pd.set_option('max_colwidth',1000)

In [3]:
import time
# time.sleep(5) # delays for 5 seconds

from sklearn.externals import joblib

In [8]:
login_name = 'name'
login_pass = 'pass'
data_df = pd.DataFrame()
db_name = "address_list1"

In [9]:
sqlite_db = db_name + '.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [10]:
col_heading = ['Icon',
'Client Reference No',
'Order Source ID',
'Original Address',
'Principal',
'Order Country',
'Location Key',
'Location Name',
'Insert Date']

In [11]:
# download chromedriver.exe and save to a local drive locaiton:
# location_of_chromedriver = "Driver/chromedriver.exe"
location_of_chromedriver = "Driver/chromedriver old.exe"
leaf_url = "https://leaf.lfuat.net/LocationManagement"

In [12]:
# set browser and go to google maps
def init_browser(URL = 'https://www.google.com.hk/maps'):
    browser = webdriver.Chrome(location_of_chromedriver)
    browser.get(URL)
    return browser

In [13]:
otm = init_browser(leaf_url)

In [14]:
login = otm.find_elements_by_class_name('form-control')
login[0].clear()
login[0].send_keys(login_name)
login[1].send_keys(login_pass)

In [15]:
# find and press the login button
btn = otm.find_elements_by_class_name('btn')
for i in btn:
    try:
        if i.text == 'Login':
            i.click()
    except:
        pass

In [16]:
def get_tables(otm, row_no=9):
    html = otm.page_source
    data_df = pd.DataFrame()
    
    bs=BeautifulSoup(html, "lxml")
    table_body=bs.find('tbody')
    rows = table_body.find_all('tr')
    tbl_result = []
    for row in rows:
        cols=row.find_all('td')
        colstrip=[x.text.strip() for x in cols]
    #     print(len(colstrip))
        tbl_result.append(colstrip)
    
    get_lst = []
    for i in tbl_result:
        if len(i) == row_no:
            get_lst.append(i)
    
    df_tmp = pd.DataFrame(get_lst)
    
    data_df = data_df.append(df_tmp)
    
    return data_df

In [17]:
def go_to_next_page():
    next_btn = otm.find_element_by_link_text('Next')
    next_btn.click()

In [18]:
def get_showing_txt():
    showing = otm.find_element_by_id('datatable_info')
    return showing.text

In [19]:
def find_last_page():
    page_buttons = otm.find_elements_by_class_name('paginate_button')
    last_page = page_buttons[-2].text
    return int(last_page)

In [16]:
find_last_page()

10

In [20]:
def get_all_addresses(set_steps=1000):
    address_df = pd.DataFrame()
    address_mapping_df = pd.DataFrame()
    
    last_page = find_last_page()
    running_int = np.min([set_steps, int(last_page)])
    for i in range(running_int):
        orig_txt = get_showing_txt()
        address_tmp = get_tables(otm, 9)
        address_tmp.columns = col_heading
        address_df = address_df.append(address_tmp)
        address_tmp.to_sql('address_list', con=conn, index=False,if_exists='append')
        
        
        for j in range(len(address_tmp)):
            # get plus buttons (again)
            loc_key = address_tmp.iloc[j]['Location Key']
            plus_btns = otm.find_elements_by_class_name('glyphicon')
            plus_btns = plus_btns[4:]
            
            # expand item
            plus_btns[j].click()
            address_map_tmp = get_tables(otm, 13)
            address_map_tmp['Source Loc'] = loc_key
            
            address_mapping_df = address_mapping_df.append(address_map_tmp)
            address_map_tmp.to_sql('address_map',con=conn,index=False,if_exists='append')
            
        new_txt = orig_txt

        go_to_next_page()

        time.sleep(1)

        counter = 0
        while new_txt == orig_txt and counter < 10:
            try:
                new_txt = get_showing_txt()
                counter = counter + 1
                time.sleep(5)
            except:
                time.sleep(5)
    address_df.columns = col_heading
    return address_df, address_mapping_df

In [22]:
# address_df = pd.read_sql('select * from address_list', con = conn)
# address_mapping_df = pd.read_sql('select * from address_map', con = conn)

In [21]:
address_df, address_mapping_df = get_all_addresses(1)
# address_df.to_excel('page_loc_table1.xlsx', index=False)
# address_mapping_df.to_excel('page_loc_mapping1.xlsx', index=False)

C:\Users\bryantlee\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [ ]:
address_df.to_excel('page_loc_table1.xlsx', index=False)
address_mapping_df.to_excel('page_loc_mapping1.xlsx', index=False)

### Automation starts here

In [20]:
loc_lookup = pd.read_excel('Lookup.xlsx')

In [21]:
plus_btns = otm.find_elements_by_class_name('glyphicon-plus-sign')

In [33]:
loc_next = otm.find_elements_by_class_name('fancybox-media')
loc_next
keep = []
for i in loc_next:
    if i.text == 'Next':
        keep.append(i)
loc_next = keep


In [35]:
loc_next[-1].click()

In [22]:
def find_and_click_next(apply_loc=-1):
    loc_next = otm.find_elements_by_class_name('fancybox-media')
    loc_next
    keep = []
    for i in loc_next:
        if i.text == 'Next':
            keep.append(i)
    loc_next = keep
    loc_next[apply_loc].click()

In [23]:
def close_iframe():
    otm.switch_to_default_content()
    close_btn = otm.find_elements_by_tag_name('a')
    keep = ''
    for i in close_btn:
        if i.get_attribute('title') == 'Close':
            keep = i
        close_btn = keep
    close_btn.click()

In [24]:
def click_ok_button():
    otm.switch_to_default_content()
    otm.switch_to_frame(otm.find_element_by_tag_name("iframe"))
    successful_btn = otm.find_elements_by_class_name('ng-binding')
    keep = []
    for i in successful_btn:
        if i.text == 'OK' or i.text == 'GOT IT':
            keep = i
    successful_btn = keep
    successful_btn.click()

In [25]:
def keep_trying(function_loop = click_ok_button, max_run=10):
    counter = 0
    found_it = False
    while found_it == False and counter < max_run:
        try:
            function_loop()
            found_it = True
        except:
            time.sleep(2)
        counter = counter + 1

In [26]:
def click_submit_button():
    otm.switch_to_default_content()
    otm.switch_to_frame(otm.find_element_by_tag_name("iframe"))
    submit_btn = otm.find_elements_by_tag_name('button')
    if submit_btn[0].text == 'SUBMIT':
        iframe_ok = True
        print('clicked')
        # click the submit button
        submit_btn[0].click()

In [27]:
def map_location_id(loc_key):
    # map the location ID to base locations, if available
    if loc_key in list(loc_lookup['OTM_LOCATION_ID']):
        tmp = loc_lookup[loc_lookup['OTM_LOCATION_ID'] == loc_key]
        map_loc_key = list(tmp['Base_LOCATION_MAPPING'])[0]
    else:
        map_loc_key = ''
    
    print(loc_key)
    print(map_loc_key)
    return map_loc_key

In [30]:
def clean_page(fix_chosen=False, clean_count = 1000):
    
    try:
        i_counter_df = pd.read_csv('i_counter.csv')
        i_counter = i_counter_df['count'].iloc[0]
    except:
        i_counter=0    

    current_page_df = get_tables(otm, row_no=9)
    current_page_df.columns=col_heading
    get_opt_list = []
    get_map_list = []
    for i in range(min(len(current_page_df),clean_count)):
        if i >= i_counter:
            print(i)

            # try to find the minus button and click it
            try:
                minus_btn = otm.find_element_by_class_name('glyphicon-minus-sign')
                minus_btn.click()
            except:
                pass

            # get plus buttons (again)
            plus_btns = otm.find_elements_by_class_name('glyphicon')
            plus_btns = plus_btns[4:]

            # map the location ID to base locations, if available
            loc_key = current_page_df.iloc[i]['Location Key']

            if loc_key in list(loc_lookup['OTM_LOCATION_ID']):
                tmp = loc_lookup[loc_lookup['OTM_LOCATION_ID'] == loc_key]
                map_loc_key = list(tmp['Base_LOCATION_MAPPING'])[0]
            else:
                map_loc_key = ''
            print(loc_key)
            print(map_loc_key)



            # if location key was valid, perform mapping
            if map_loc_key != "":

                # expand item
                plus_btns[i].click()

                # check the item is expanded
                print('expand list')
                expanded = False
                counter = 0
                while expanded == False and counter < 10:
                    time.sleep(1)
                    try:
                        tms_drop_down = otm.find_elements_by_class_name('form-control')
                        expanded = True
                    except:
                        time.sleep(2)
                    counter = counter + 1

                # get status
                open_df = get_tables(otm, 13)
                get_option = False
                get_mapping = False
                try:
                    opt_lst = list(open_df[open_df[11] == map_loc_key].index)

                    loc_check = open_df[["LOC" in i for i in list(open_df[11]) ]]
                    if len(loc_check) > 0:
                        get_option = True
                    chosen = np.min(opt_lst)
                    if len(opt_lst) > 0:
                        get_mapping = True
                except:
                    chosen = -1

#                 if fix_chosen:
#                     chosen = -1

                get_opt_list.append(get_option)
                get_map_list.append(get_mapping)



                if chosen != -1 or fix_chosen==True:

                    if get_mapping == False:
                        # send the location key to text box
                        time.sleep(1)
                        tms_drop_down = otm.find_elements_by_class_name('form-control')
                        tms_drop_down[0].send_keys('TMS')

                        location_id_value = otm.find_element_by_id('locationID_value')
                        location_id_value.clear()
                        location_id_value.send_keys(map_loc_key)

                        # press down button
                        time.sleep(3)
                        location_id_value.send_keys(Keys.DOWN)

                        # press enter
                        time.sleep(0.5)
                        location_id_value.send_keys(Keys.RETURN)

                    # click next button
                    print('click next')

                    time.sleep(1)
                    print(chosen)
                    find_and_click_next(chosen)

                    # check the iframe is up and submit
                    print('submit')
                    time.sleep(6)
                    keep_trying(click_submit_button)

                    time.sleep(1)
                    print('click ok')
                    # click ok
                    keep_trying(click_ok_button)

                    time.sleep(1)
                    print('close iframe')
                    # close iframe
                    keep_trying(close_iframe,1)

                    time.sleep(1)

                # minimize the expanded cell
                try:
                    minus_btn = otm.find_element_by_class_name('glyphicon-minus-sign')
                    minus_btn.click()
                except:
                    pass

        clear_output()
        i_counter = i
        i_counter_df = pd.DataFrame()
        i_counter_df['count'] = [i_counter]
#         i_counter_df.to_csv('i_counter.csv',index=False)        

    #     print(get_opt_list)
    #     print(get_map_list)
    return get_opt_list, get_map_list        
            

In [31]:
option_status, mapping_status = clean_page(fix_chosen=False)

0
0424LOC0034
0424LOC1852
expand list
click next
-1
submit
click ok


KeyboardInterrupt: 

In [34]:
current_btn = otm.find_element_by_class_name('current')

In [35]:
current_btn.text

'1'

In [36]:
def page_clean(fix_chosen):
        
    page_buttons = otm.find_elements_by_class_name('paginate_button')
    last_page = page_buttons[-2].text
    print(last_page)

    current_btn = otm.find_element_by_class_name('current')
    current_no = int(current_btn.text)
    last_no = int(last_page)

    page_flip = last_no - current_no + 1

    address_df = pd.DataFrame()
    for i in range(page_flip):
        print('Cleaning {} page of {} pages'.format(str(current_no), last_page))
        orig_txt = get_showing_txt()
        option_status, mapping_status = clean_page(fix_chosen=fix_chosen)

        time.sleep(2)

        go_to_next_page()

        counter = 0
        time.sleep(5)
        new_txt = ""
        while new_txt == orig_txt and counter < 10:
            try:
                new_txt = get_showing_txt()
                counter = counter + 1
            except:
                time.sleep(5)

In [40]:
ending_status = False

while ending_status == False:

    try:
        page_clean(fix_chosen=False)
    except:
        time.sleep(10)
        
    current_btn = otm.find_element_by_class_name('current')
    if int(current_btn.text) >= find_last_page():
        ending_status = True

8
0424LOC0751
0424LOC2548


In [ ]:
page_clean(fix_chosen=False)

In [40]:
otm.close()